In [1]:
import json
import pymongo
from pymongo.database import Database
from pymongo.collection import Collection
from time import time
import datetime
with open("watermelon.config", "rb") as f:
    js = json.load(f)
    mongo_key: str = js["mongo_key"]
    prefix: str = js["prefix"]

if prefix in ["w?", "t?"]:  # only access mongodb for w? and t?
    client = pymongo.MongoClient(mongo_key)
    db: Database = client.get_database("AlexMindustry")
    expgains: Collection = db["expgains"]
    convertedexp: Collection = db["convertedexp"]
    muuid: Collection = db["muuid"]
    # V7 stuff
    expv7: Collection = db["expv7"]
    convertedexpv7: Collection = db["convertedexpv7"]
    ingamecosmeticsv7: Collection = db["ingamecosmeticsv7"]
    serverplayerupdates1: Collection = db["serverplayerupdates1"]
    hexv7: Collection = db["hexdataV7"]

## this notebook filters documents with duplicated muuid,usid,ip. and keeps the unique one with the latest date.

## to clean up, run the first 3 cells

In [2]:
def clean_up_duplicate_muuid(timedeltadays=3,num_of_days_in_the_past=100):
    for i in range(0,num_of_days_in_the_past//timedeltadays):
        start_date = datetime.datetime.now() - datetime.timedelta(days=(i+1)*timedeltadays)
        end_date = datetime.datetime.now() - datetime.timedelta(days=i*timedeltadays)
        current_date = start_date
        would_delete = []
        while current_date < end_date:
            next_date = current_date + datetime.timedelta(days=timedeltadays)
            # Group documents by unique fields excluding _id and date
            pipeline = [
                {
                    '$match': {
                        'date': {'$gte': current_date, '$lt': next_date}
                    }
                },
                {
                    '$group': {
                        '_id': {
                            'musername': '$musername',
                            'muuid': '$muuid',
                            'musid': '$musid',
                            'con_address': '$con_address',
                            'color': '$color',
                            'servername': '$servername'
                        },
                        'latest': {'$max': '$date'},
                        'docs': {'$push': {'_id': '$_id', 'date': '$date'}}
                    }
                }
            ]
            # Execute the aggregation pipeline
            result = muuid.aggregate(pipeline)
            # Loop through the aggregation result
            keep=0
            for doc in result:
                latest_date = doc['latest']
                docs = doc['docs']
                latest_id = None
                # Find the document with the latest date
                for d in docs:
                    if d['date'] == latest_date:
                        latest_id = d['_id']
        
                # Print all other documents except the latest one
                if latest_id:
                    keep+=1
                    for d in docs:
                        if d['_id'] != latest_id:
                            #print(f"Would delete document: {d['_id']}")
                            would_delete.append(d['_id'])
            print("docs",len(would_delete),"keep",keep,next_date)
            current_date = next_date
        res = muuid.delete_many({'_id': {'$in': would_delete}})
        print("deleted",res.deleted_count)
    print("Run complete!")

In [3]:
%%time
clean_up_duplicate_muuid(timedeltadays=3,num_of_days_in_the_past=100)
clean_up_duplicate_muuid(timedeltadays=10,num_of_days_in_the_past=100)
clean_up_duplicate_muuid(timedeltadays=30,num_of_days_in_the_past=100) # max is 50.

docs 4851 keep 13202 2024-08-31 12:45:50.347515
deleted 4851
docs 5718 keep 9507 2024-08-28 12:45:58.367481
deleted 5718
docs 7037 keep 10959 2024-08-25 12:46:05.301512
deleted 7037
docs 6420 keep 10272 2024-08-22 12:46:12.240885
deleted 6420
docs 7706 keep 11394 2024-08-19 12:46:20.774229
deleted 7706
docs 6325 keep 10604 2024-08-16 12:46:27.755992
deleted 6325
docs 6747 keep 10845 2024-08-13 12:46:33.688744
deleted 6747
docs 7036 keep 12415 2024-08-10 12:46:42.258968
deleted 7036
docs 6154 keep 12098 2024-08-07 12:46:49.234460
deleted 6154
docs 7395 keep 11839 2024-08-04 12:46:57.056684
deleted 7395
docs 4999 keep 8019 2024-08-01 12:47:05.999278
deleted 4999
docs 8545 keep 11988 2024-07-29 12:47:13.997338
deleted 8545
docs 8257 keep 12106 2024-07-26 12:47:21.902220
deleted 8257
docs 8459 keep 14203 2024-07-23 12:47:28.411793
deleted 8459
docs 8260 keep 16704 2024-07-20 12:47:36.659434
deleted 8260
docs 3878 keep 14268 2024-07-17 12:47:45.813568
deleted 3878
docs 0 keep 14634 2024-07-

# delete old docs in playerserverupdates1

# this will delete incrementally in batches of 10 days

In [18]:
print("DRY RUN")
num_days_ago = 30 # delete docs older than X days.
for num_days_ago in list(range(90,5,-10)):
    days_ago = datetime.datetime.now() - datetime.timedelta(days=num_days_ago)
    filterr = {'date': {'$lt': days_ago}}
    # Count the number of documents that would be deleted
    count = serverplayerupdates1.count_documents(filterr)
    print(f"Number of documents to be deleted: {count}")

dry run
Number of documents to be deleted: 0
Number of documents to be deleted: 0
Number of documents to be deleted: 0
Number of documents to be deleted: 0
Number of documents to be deleted: 0
Number of documents to be deleted: 0
Number of documents to be deleted: 0
Number of documents to be deleted: 73584
Number of documents to be deleted: 143290


In [19]:
print("ACTUAL RUN")
num_days_ago = 30 # delete docs older than X days.
for num_days_ago in list(range(90,5,-10)):
    days_ago = datetime.datetime.now() - datetime.timedelta(days=num_days_ago)
    filterr = {'date': {'$lt': days_ago}}
    # Count the number of documents that would be deleted
    count = serverplayerupdates1.count_documents(filterr)
    print(f"Number of documents to be deleted: {count}")
    result = serverplayerupdates1.delete_many(filterr)
    print(f"Deleted {result.deleted_count} documents.")

actual run
Number of documents to be deleted: 0
Deleted 0 documents.
Number of documents to be deleted: 0
Deleted 0 documents.
Number of documents to be deleted: 0
Deleted 0 documents.
Number of documents to be deleted: 0
Deleted 0 documents.
Number of documents to be deleted: 0
Deleted 0 documents.
Number of documents to be deleted: 0
Deleted 0 documents.
Number of documents to be deleted: 8
Deleted 8 documents.
Number of documents to be deleted: 73594
Deleted 73594 documents.
Number of documents to be deleted: 69697
Deleted 69697 documents.


# END OF FILE

In [15]:
%%time
num_days_ago = 30 # delete docs older than X days.
days_ago = datetime.datetime.now() - datetime.timedelta(days=num_days_ago)
filterr = {'date': {'$lt': days_ago}}
# Count the number of documents that would be deleted
count = serverplayerupdates1.count_documents(filterr)
print(f"Number of documents to be deleted: {count}")

Number of documents to be deleted: 80007
CPU times: total: 15.6 ms
Wall time: 411 ms


In [17]:
list(range(90,5,-10))

[90, 80, 70, 60, 50, 40, 30, 20, 10]

In [16]:
%%time
result = serverplayerupdates1.delete_many(filterr)
print(f"Deleted {result.deleted_count} documents.")

Deleted 80007 documents.
CPU times: total: 0 ns
Wall time: 17.3 s


In [2]:
t1=time()
alexid=612861256189083669
xxx1=315764312700485632 #cohl
xxx2=1185756027929501718 #cen
duuid=alexid
t2=time()

In [38]:
%%time
collection = muuid

# Calculate the start date for 2 months ago
for i in range(0,4):
    start_date = datetime.datetime.now() - datetime.timedelta(days=(i+1)*40) #150 to 
    end_date = datetime.datetime.now() - datetime.timedelta(days=i*40)
    
    current_date = start_date
    
    would_delete = []
    
    while current_date < end_date:
        next_date = current_date + datetime.timedelta(days=40)
        
        # Group documents by unique fields excluding _id and date
        pipeline = [
            {
                '$match': {
                    'date': {'$gte': current_date, '$lt': next_date}
                }
            },
            {
                '$group': {
                    '_id': {
                        'musername': '$musername',
                        'muuid': '$muuid',
                        'musid': '$musid',
                        'con_address': '$con_address',
                        'color': '$color',
                        'servername': '$servername'
                    },
                    'latest': {'$max': '$date'},
                    'docs': {'$push': {'_id': '$_id', 'date': '$date'}}
                }
            }
        ]
    
        # Execute the aggregation pipeline
        result = collection.aggregate(pipeline)
    
        # Loop through the aggregation result
        keep=0
        for doc in result:
            latest_date = doc['latest']
            docs = doc['docs']
            latest_id = None
    
            # Find the document with the latest date
            for d in docs:
                if d['date'] == latest_date:
                    latest_id = d['_id']
    
            # Print all other documents except the latest one
            if latest_id:
                keep+=1
                for d in docs:
                    if d['_id'] != latest_id:
                        #print(f"Would delete document: {d['_id']}")
                        would_delete.append(d['_id'])
        print("docs",len(would_delete),"keep",keep,next_date)
    
        current_date = next_date
    
    print("Run complete!")
    res = muuid.delete_many({'_id': {'$in': would_delete}})
    print("delted",res.deleted_count)

docs 690 keep 188641 2024-06-22 17:17:44.687712
Run complete!
delted 690
docs 3004 keep 157589 2024-05-13 17:20:24.273383
Run complete!
delted 3004
docs 1008 keep 158618 2024-04-03 17:22:16.759669
Run complete!
delted 1008
docs 4 keep 24270 2024-02-23 17:23:56.078185
Run complete!
delted 4
CPU times: total: 6.44 s
Wall time: 6min 32s


In [13]:
%%time
from bson.objectid import ObjectId

object_id_list = [ObjectId(idd) for idd in would_delete]
query = {'_id': {'$in': object_id_list}}

matching_documents = muuid.find(query)
counts=0
for doc in matching_documents:
    #print(doc)
    counts+=1

CPU times: total: 46.9 ms
Wall time: 7.6 s


In [14]:
counts

13346